# 各サンプルにおけるベイズ最適性の確認

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from tqdm import tqdm
from scipy.stats import entropy
from sklearn.linear_model import LogisticRegression
from polyagamma import random_polyagamma
from utils.generate import generate_data
from utils.metrics import KLD
from models.proposed_method import proposed_method
from models.compared_methods import compared_methods

ImportError: cannot import name 'generate_data' from 'utils.generate' (/Users/abetaichi/master/Lab/research/selection_bias/for_paper/src/utils/generate.py)

In [88]:
import warnings
warnings.filterwarnings('ignore')

In [55]:
# パラメータ設定
n_samples = 100  # サンプルサイズ
n_features = 1  # 特徴量数（切片除く）
X_mu_list = [10]  # 各特徴量の平均
X_sigma_list = [10]  # 各特徴量の標準偏差
beta_mu_list = [1, 1]  # パラメータの平均 (切片を含む)
beta_sigma_list = [0.1, 0.1]  # パラメータの標準偏差 (切片を含む)

X_feature_bias = 1 # 0は切片
center_of_sampling = 0
window_size = 3

# データ生成
data_generator = generate_data(n_features, X_mu_list, X_sigma_list, beta_mu_list, beta_sigma_list)
X, y = data_generator.generate_non_bias_data()
X_bias, y_bias = data_generator.generate_bias_data_deterministic(n_samples, X_feature_bias, center_of_sampling, window_size)
P_true = y.mean()

# P_yの推定
b_0 = np.array([0.001,0.001])
B_0 = np.array([[100,0],
                [0,100]])
burn=5000
draw=10000

proposed_model = proposed_method(X=X_bias, y=y_bias, b_0=b_0, B_0=B_0, X_mu=X_mu_list, X_sigma=X_sigma_list, burn=burn, draw=draw)
_ = proposed_model.sample_beta() # バーンインも含まれるので注意
P_y_proposed = proposed_model.estimate()

compared_model = compared_methods(X=X_bias, y=y_bias, X_mu=X_mu_list, X_sigma=X_sigma_list)
P_y_sample_mean = compared_model.sample_mean()
P_y_ML = compared_model.maximum_likelihood(draw)

print(f"真のP_y: {P_true}")
print(f"提案手法によるP_y: {P_y_proposed}")
print(f"標本平均によるP_y: {P_y_sample_mean}")
print(f"最尤法によるP_y: {P_y_ML}")
KLD(P_true, P_y_ML)

真のP_y: 0.8578152
提案手法によるP_y: 0.8629875505564821
標本平均によるP_y: 0.72
最尤法によるP_y: 0.8429683287786903


np.float64(0.0008787283720850247)

In [89]:
# パラメータを母集団分布から生成してKL計算して...を指定の回数繰り返す
## 実験パターン：Xの母平均，サンプリング位置

Ex_num = 100 # パラメータを生成する回数
X_mu_cands = [-5,-2,0,2,5] # Xの母平均リスト
Center_num = 5 # サンプリング位置（個数を指定して自動生成）


# 実験設定（全パターン共通）
n_samples = 100  # サンプルサイズ
n_features = 1  # 特徴量数（切片除く）
X_sigma_list = [10]  # 各特徴量の標準偏差
beta_mu_list = [0, 1]  # パラメータの平均 (切片を含む)
beta_sigma_list = [1, 1]  # パラメータの標準偏差 (切片を含む)

X_feature_bias = 1 # 0は切片
window_size = X_sigma_list[0] / 2

# P_yの推定
b_0 = np.array([0.001,0.001])
B_0 = np.array([[100,0],
                [0,100]])
burn=2000
draw=5000

for X_mu in X_mu_cands: # Xの母平均を変更
    #lower = X_mu - X_sigma_list[0] / 2
    #upper = X_mu + X_sigma_list[0] / 2
    lower = -5
    upper = 5
    center_list = np.linspace(lower, upper, Center_num)
    KL_list_proposed = []
    KL_list_sample_mean = []
    KL_list_ML = []

    for center in center_list: # サンプリング位置を変更
        KL_list_proposed_by_center = []
        KL_list_sample_mean_by_center = []
        KL_list_ML_by_center = []
        # 結果の表示
        print("-----------start-----------")
        print(f"共変量の母平均:{X_mu}")
        print(f"サンプリングの範囲{center-window_size}〜{center+window_size}")
        print("---------------------------")

        trial = 0 # KL計算に成功した回数（0と1を両方含むデータ生成に成功した回数）→Ex_numに到達したら終わり
        with tqdm() as pbar:
            while trial < Ex_num: # パラメータによるモンテカルロ平均（ベイズ最適な推定量）
                # データの生成
                X_mu_list = [X_mu] # リスト形式なので，リストに変更（あとで修正）
                data_generator = generate_data(n_features, X_mu_list, X_sigma_list, beta_mu_list, beta_sigma_list)
                X, y = data_generator.generate_non_bias_data()
                X_bias, y_bias = data_generator.generate_bias_data_deterministic(n_samples, X_feature_bias, center, window_size)

                # KLの計算
                P_true = y.mean() # 真のP
                P_bias_true = y_bias.mean()

                if (P_bias_true != 1) & (P_bias_true != 0):

                    proposed_model = proposed_method(X=X_bias, y=y_bias, b_0=b_0, B_0=B_0, X_mu=X_mu_list, X_sigma=X_sigma_list, burn=burn, draw=draw)
                    _ = proposed_model.sample_beta() # バーンインも含まれるので注意
                    P_y_proposed = proposed_model.estimate() # 提案によるP

                    compared_model = compared_methods(X=X_bias, y=y_bias, X_mu=X_mu_list, X_sigma=X_sigma_list)
                    P_y_sample_mean = compared_model.sample_mean() # 標本平均によるP
                    P_y_ML = compared_model.maximum_likelihood(draw) # 最尤推定によるP

                    KL_list_proposed_by_center.append(KLD(P_true, P_y_proposed))
                    KL_list_sample_mean_by_center.append(KLD(P_true, P_y_sample_mean))
                    KL_list_ML_by_center.append(KLD(P_true, P_y_ML))

                    pbar.update(1)
                    trial += 1
                else:
                    pass
        
        print(f"Yの母平均:{np.mean(y)}")
        print(f"提案の平均KL:{np.mean(KL_list_proposed_by_center)}")
        print(f"標本平均の平均KL:{np.mean(KL_list_sample_mean_by_center)}")
        print(f"最尤推定の平均KL:{np.mean(KL_list_ML_by_center)}")
    
        KL_list_proposed.append(np.mean(KL_list_proposed_by_center))
        KL_list_sample_mean.append(np.mean(KL_list_sample_mean_by_center))
        KL_list_ML.append(np.mean(KL_list_ML_by_center))

    # グラフ描画
    plt.plot(center_list, KL_list_proposed, label="Proposed method", color = "red",marker='*',markersize=12)
    plt.plot(center_list, KL_list_sample_mean, label="sample mean", color = "blue",marker='.',markersize=12)
    plt.plot(center_list, KL_list_ML, label="Maximum Likelihood", color = "green",marker='o',markersize=12)

    plt.title(f"X_mu:{X_mu}, X_sigma:{X_sigma_list[0]}, β0:N({beta_mu_list[0]},{beta_sigma_list[0]}), β1:N({beta_mu_list[0]},{beta_sigma_list[0]})", fontsize = 12)
    plt.xlabel("center of sampling", fontsize = 12)
    plt.ylabel("BayesRisk(Average KL Divergence)", fontsize = 12)
    plt.xlim(lower-0.5, upper+0.5)
    plt.legend(bbox_to_anchor=(0.99, 0.99), loc='upper right', borderaxespad=0, fontsize=12)

    save_dir = "../results/fig"
    plt.savefig(f"{save_dir}/BayesRisk_Xmu_{X_mu}.png")
    plt.close()

-----------start-----------
共変量の母平均:-5
サンプリングの範囲-10.0〜0.0
---------------------------


100it [04:16,  2.56s/it]


Yの母平均:0.7364772
提案の平均KL:0.008560724197854521
標本平均の平均KL:0.18035477288776325
最尤推定の平均KL:0.0153282267714156
-----------start-----------
共変量の母平均:-5
サンプリングの範囲-7.5〜2.5
---------------------------


100it [03:12,  1.92s/it]


Yの母平均:0.615609
提案の平均KL:0.003896342526501822
標本平均の平均KL:0.10654583307091794
最尤推定の平均KL:0.0063520816719622245
-----------start-----------
共変量の母平均:-5
サンプリングの範囲-5.0〜5.0
---------------------------


...

Yの母平均:0.6938805
提案の平均KL:0.005314140884116838
標本平均の平均KL:0.07259859220656542
最尤推定の平均KL:0.008601833138676629
-----------start-----------
共変量の母平均:5
サンプリングの範囲-2.5〜7.5
---------------------------


100it [01:58,  1.19s/it]


Yの母平均:0.7132894
提案の平均KL:0.002720231572205796
標本平均の平均KL:0.09350293914119344
最尤推定の平均KL:0.0071510518271539845
-----------start-----------
共変量の母平均:5
サンプリングの範囲0.0〜10.0
---------------------------


100it [02:48,  1.69s/it]

Yの母平均:0.6663977
提案の平均KL:0.009285538833194839
標本平均の平均KL:0.15921833966424434
最尤推定の平均KL:0.012947286427712552


In [90]:
17/43 - 13/43

0.09302325581395349